# <span style="color:darkred">Data extract, manipulation and visualization</span>
Developer: Henriette Steenhoff



### <span style="color:darkred">Providers of data and recipients of result</span>
#### Jesper Bo Sembach, Niels Beuschau
*Master Thesis in Internet Security and Ransomware Detection*<br></br>
The Technical University of Denmark (DTU), June 2017

#### <span style="color:darkred">Purpose</span>
This code was made for extracting important data for analysis and results for the Master Thesis of Jesper Bo Sembach and Niels Beuschau. The code extracts and manipulates data from their test environment into correct data structures for visualization and performance analysis. 

----

### <span style="color:darkred">The code</span>

#### <span style="color:darkred">Functions</span>
For code ledgibility

In [47]:
# -------------------------------------------------------------------------------- #
# The Technical University of Denmark, June 2017
# Developer: Henriette Steenhoff, s134869
# Purpose:   Analysis and visualization of performance measures for Master Thesis of 
#            Jesper Bo Sembach and Niels Beuschau
# -------------------------------------------------------------------------------- #

# IMPORTS
import re
import json
# Only importing needed modules to avoid os.open being chosen instead of built-in open
from os import path, chdir, makedirs, listdir, getcwd 
import time
import matplotlib as plt
%matplotlib inline

import plotly
import plotly.plotly as py
from IPython.display import Image 
import plotly.graph_objs as go
# API access to plotting tools
plotly.tools.set_credentials_file(username='frksteenhoff2', api_key ='duu8hsfRmuI5rF2EU8o5')

basePath    = "C:/Users/frksteenhoff/Documents/GitHub/MSc-Sembach-Beuschau_Jun2017/"
pathToPlots = basePath + "Visualization"

# -------------------------------------------------------------------------------- #

# HELPER FUNCTIONS FOR THE VISUALIZATION SCRIPT BELOW

# Function creating correct layout style for each type of plot 
def setLayoutOptions(headTitle, ytitle):
    layout= go.Layout(
            title= headTitle.upper(),
            hovermode= 'closest',
            xaxis= dict(
                title= 'Time (0.5 second interval)',
                ticklen= 5,
                zeroline= True,
                gridwidth= 2,
            ),
            yaxis=dict(
                title= ytitle,
                ticklen= 5,
                gridwidth= 2,
            ),
            showlegend= False
    )
    return layout


# Remove all occurences of AppData paths from a given list of paths
def removeAppData(lst):
    outFile = []
    for f in lst:
        current_file = f.split('*')[0]
        pattern = re.compile('.*AppData.*')
        if not pattern.match(current_file):
            outFile.append(f)
    return outFile


# Split information from txt file in title and data
def splitTitleAndFiles(line, lineNumber):
    temp = []
    separteValues = []
    
    temp = line.strip('\n')
    titleAndList = temp.split(';')
    title = titleAndList[0]
    print title
        
    separateValues = titleAndList[1].split('?')
     
    if lineNumber in range(11,16):
        values = [val.replace(',','.') for val in separateValues]
    else:
        values = separateValues
    
    return title, values

    
# Create cpu, ram, hdd, handle and thread count plots
def createPerformancePlots(title, listValues, lineNumber, filename, folder, pathToPlots, pathToFiles):
    mode = 'lines'
    
    # Set plotting marker
    #if lineNumber == 11:
    #    mode = 'markers'
    #else:
    #    mode = 'lines'
        
    trace1 = go.Scatter(
        x = range(0,len(listValues)),
        y = listValues, 
        mode = mode,
        marker = dict(
            size = 3
        )
    )    
    
    # Choose right values for plotting cpu, ram or hdd, handle and thread count
    if lineNumber == 11:
        layout = setLayoutOptions(title, 'Percent')
    if lineNumber == 12:
        layout = setLayoutOptions(title, 'MB')
    elif lineNumber == 13:
        layout = setLayoutOptions(title, 'Speed - missing unit')
    elif lineNumber == 14:
        layout = setLayoutOptions(title, 'Number of handles')
    elif lineNumber == 15:
        layout = setLayoutOptions(title, 'Number of threads')
    else:    
        layout = setLayoutOptions(title, 'Byte')

    data = [trace1]
    fig  = go.Figure(data=data, layout=layout)
    
    # Create folder for plots if not exists
    dir_name = pathToPlots + "/" + folder
    if not path.exists(dir_name):
        makedirs(dir_name)
    chdir(dir_name)
    
    # Save image to file
    py.image.save_as(fig, filename=filename.strip('.txt') + "-"+ title + '.png')    
    chdir(pathToFiles)
    time.sleep(2)
    
    
# ('File sizes over time', fileSizes, files, pathToPlots, pathToFiles)
def createFileSizePlots(title, size_list, filename, folder, plot_path, file_path):
    dir_name = plot_path + "/" + folder
    chdir(dir_name)
    
    plt.scatter(range(0,len(size_list)), size_list, '-o', alpha=0.5)
    plt.title(title)
    plt.xlabel('Byte')
    plt.ylabel('Time (0.5 second interval)')
    plt.savefig(title + '-size.png')
    chdir(file_path)

def findFilePath(fileList, splitCrit):
    fileNames = []
    
    for f in fileList:
        pathStr = f.split(splitCrit)[0]
        fileNames.append(re.sub(r'\D:\\{1,4}\w*\\{1,4}\w*\\{1,4}', '', pathStr))
    return fileNames

        
def getFileSizes(filename):
    sizeOfFiles = {}
    # Read each line and split content into dictionary
    chdir(basePath)
    with open(filename, 'r') as f:
        paths = f.readlines()
    chdir(pathToFiles)
    
    for lines in paths:
        values = lines.split('?')
        keyy = values[0].strip("C:\Users\Baseline\\")
        sizeOfFiles[keyy] = values[1]                                   
    return sizeOfFiles

                                   
# Here assuming that all files can be found in size_dict (from baselinedata.txt)                           
def createFileSizeList(files, size_dict):
    sizeList = []
    for f in files:
        # Strip f of excess backslashes
        keyy = re.sub(r'\\{2}','\\\\', f)
        sizeList.append(size_dict[keyy])
    return sizeList
                                   

#### <span style="color:darkred">Read all files in each folder of "Speciale"</span>
All data from folder ``Speciale``, found [here](www.github.com/frksteenhoff/MSc-Niels-Jesper-June2017/) on Github will, be used for extracting visualizations. The visualizations will be placed in a separate folder with the same structure as the ``Speciale`` folder, named ``Visualization``.

In [48]:
# -------------------------------------------------------------------------------- #
# The actual visualization script
# -------------------------------------------------------------------------------- #

# Question: why are the shannon and honey pot number not the same?
folders = ['baseline', 'hp1', 'hp2','hp5', 'hp10', 'sh3', 'sh5', 'sh10', 'sh15']
lineNumber = 0
obsMon = []
fileSizes = []
baselineData = getFileSizes('BaselineFileData.txt')

# For all folders in Speciale, access each folder and  'do something'
for folder in folders[3:]:
    pathToFiles   = basePath + "Speciale/" + folder 
    chdir(pathToFiles)
    selectedFiles = [file for file in listdir('.') if file.endswith('.txt')]
    print "Current folder: " + folder
    
    # For each file in current folder, read content 
    for files in selectedFiles:
        print files, lineNumber
        
        # Read each line and split content into dictionary
        with open(files, 'r') as f:
            content = f.readlines()
        
        for lines in content:
            # Noting implemented on first information in file yet                
            if lineNumber in range(0,11):
                print lines.split(';')[0].ljust(24), "not implemented"
                  
            # If we have data for cpu, ram, hdd, thread or handle count 
            elif lineNumber in range(11,16) and lineNumber == 1:
                # Get attribute and list of values
                title, listValues = splitTitleAndFiles(lines, lineNumber)
                    
                # Create plots cpu, ram, hdd, handle and thread count
                createPerformancePlots(title, listValues, lineNumber, files, folder, pathToPlots, pathToFiles)
               
            # If fileMonObservations
            elif (lineNumber == 19):
                # Get attribute and list of values
                title, listValues = splitTitleAndFiles(lines, lineNumber)
                    
                # Extract all files not in folder AppData
                obsMon = removeAppData(listValues)
                filePaths = findFilePath(obsMon, '*')
                fileSizes = createFileSizeList(filePaths, baselineData)
                createFileSizePlots('File sizes over time', fileSizes, 0, files, pathToPlots, pathToFiles)
            
            # Nothing implemented on last information in file yet 
            else:
                print lines.strip('\n').split(';')[0].ljust(24), "not implemented"
            lineNumber += 1
        lineNumber = 0
        fileSizes = []        

print "\nAll files read"

Current folder: hp5
00931cd1bcb134399acfde7870cf110477707bed747f0d9895b0b8cc361334bb.txt 0
ransomware
              not implemented
ransomware               not implemented
fecthed                  not implemented
started                  not implemented
posted                   not implemented
monitorStatus            not implemented
monitorCount             not implemented
countChangedFiles        not implemented
countDeletedFiles        not implemented
countNewFiles            not implemented
countFilemonObservations not implemented
cpu                      not implemented
ram                      not implemented
hdd                      not implemented
threadCount              not implemented
handleCount              not implemented
listChangedFiles         not implemented
listDeletedFiles         not implemented
listNewFiles             not implemented
listFilemonObservations


KeyError: 'Desktop\\ransomware.exe'

In [50]:
baselineData

{'Desktop\\decided.docx': '1510988',
 'Desktop\\Modul 1 -- M\xc3\xa6ngder og relationer\\uge3.pdf': '43320',
 'Videos\\Ovelser\\140912 IFVT2 2 ovelser.pdf': '24012',
 'Desktop\\a downon.docx': '640592',
 'Desktop\\lying and.docx': '1373342',
 'Desktop\\a  into.docx': '244009',
 'Downloads\\Java\\Ugesedler mm\\Uge 11 14-04-2011\\11slidesF.pdf': '618911',
 'Desktop\\vendor\\slim\\slim\\Slim\\Exception\\ContainerValueNotFoundException.php': '502',
 'Downloads\\Hedwig. a his.docx': '171102',
 'Desktop\\Losninger til grupperegning\\week11.pdf': '256801',
 'Desktop\\Eksamen\\Eksamen10022F09Loesning.pdf': '111478',
 'Documents\\cn (41).zip': '23429287',
 'Documents\\said helped.docx': '1519376',
 'Downloads\\Google\\Harry crowd around. hes to came.docx': '1026323',
 "Downloads\\Exercises\\Module 12 -- Last Semester's Exam (No Hand In)\\ex02-main.cpp": '402',
 'Downloads\\SkovBrandPub_10.pdf': '314363',
 'Desktop\\Mozilla Firefox\\Workshop presentations\\Privacy in the Online Social Network (s

----
### Shortcut to simple styling tricks
``<span style="color:darkred"></span>``